In [14]:
import os
import sys
import matplotlib

import matplotlib.pyplot
import matplotlib.pyplot as plt
import time
import datetime
import argparse
import numpy as np
import pandas as pd
from random import SystemRandom
from sklearn import model_selection
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from torch.nn.functional import relu
import torch.optim as optim

import lib.utils as utils
from lib.plotting import *

from lib.rnn_baselines import *
from lib.ode_rnn import *
from lib.create_latent_ode_model import create_LatentODE_model
from lib.parse_datasets import parse_datasets
from lib.ode_func import ODEFunc, ODEFunc_w_Poisson
from lib.diffeq_solver import DiffeqSolver
from mujoco_physics import HopperPhysics
import os
import numpy as np

import torch
import torch.nn as nn

import lib.utils as utils
from lib.diffeq_solver import DiffeqSolver
from generate_timeseries import Periodic_1d
from torch.distributions import uniform

from torch.utils.data import DataLoader
from mujoco_physics import HopperPhysics
from physionet import variable_time_collate_fn, get_data_min_max
from person_activity import PersonActivity, variable_time_collate_fn_activity

from sklearn import model_selection
import random
from lib.utils import compute_loss_all_batches

In [15]:
# Generative model for noisy data based on ODE
parser = argparse.ArgumentParser('Latent ODE')
parser.add_argument('-n',  type=int, default=10000, help="Size of the dataset")
parser.add_argument('--niters', type=int, default=20)
parser.add_argument('--lr',  type=float, default=1e-2, help="Starting learning rate.")
parser.add_argument('-b', '--batch-size', type=int, default=256)
parser.add_argument('--viz', action='store_true', default=False, help="Show plots while training")

parser.add_argument('--save', type=str, default='experiments/', help="Path for save checkpoints")
parser.add_argument('--load', type=str, default=None, help="ID of the experiment to load for evaluation. If None, run a new experiment.")
parser.add_argument('-r', '--random-seed', type=int, default=1991, help="Random_seed")

parser.add_argument('--dataset', type=str, default='physionet', help="Dataset to load. Available: physionet, activity, hopper, periodic")
parser.add_argument('-s', '--sample-tp', type=float, default=None, help="Number of time points to sub-sample."
	"If > 1, subsample exact number of points. If the number is in [0,1], take a percentage of available points per time series. If None, do not subsample")

parser.add_argument('-c', '--cut-tp', type=int, default=None, help="Cut out the section of the timeline of the specified length (in number of points)."
	"Used for periodic function demo.")

parser.add_argument('--quantization', type=float, default=5, help="Quantization on the physionet dataset."
	"Value 1 means quantization by 1 hour, value 0.1 means quantization by 0.1 hour = 6 min")

parser.add_argument('--latent-ode', action='store_true', default=True,  help="Run Latent ODE seq2seq model")
parser.add_argument('--z0-encoder', type=str, default='odernn', help="Type of encoder for Latent ODE model: odernn or rnn")

parser.add_argument('--classic-rnn', action='store_true', help="Run RNN baseline: classic RNN that sees true points at every point. Used for interpolation only.")
parser.add_argument('--rnn-cell', default="gru", help="RNN Cell type. Available: gru (default), expdecay")
parser.add_argument('--input-decay', action='store_true', help="For RNN: use the input that is the weighted average of impirical mean and previous value (like in GRU-D)")

parser.add_argument('--ode-rnn', action='store_true', help="Run ODE-RNN baseline: RNN-style that sees true points at every point. Used for interpolation only.")

parser.add_argument('--rnn-vae', action='store_true', help="Run RNN baseline: seq2seq model with sampling of the h0 and ELBO loss.")

parser.add_argument('-l', '--latents', type=int, default=6, help="Size of the latent state")
parser.add_argument('--rec-dims', type=int, default=40, help="Dimensionality of the recognition model (ODE or RNN).")

parser.add_argument('--rec-layers', type=int, default=3, help="Number of layers in ODE func in recognition ODE")
parser.add_argument('--gen-layers', type=int, default=3, help="Number of layers in ODE func in generative ODE")

parser.add_argument('-u', '--units', type=int, default=50, help="Number of units per layer in ODE func")
parser.add_argument('-g', '--gru-units', type=int, default=100, help="Number of units per layer in each of GRU update networks")

parser.add_argument('--poisson', action='store_true', help="Model poisson-process likelihood for the density of events in addition to reconstruction.")
parser.add_argument('--classif', action='store_true', help="Include binary classification loss -- used for Physionet dataset for hospiral mortality")

parser.add_argument('--linear-classif', action='store_true', help="If using a classifier, use a linear classifier instead of 1-layer NN")
parser.add_argument('--extrap', action='store_true', help="Set extrapolation mode. If this flag is not set, run interpolation mode.")

parser.add_argument('-t', '--timepoints', type=int, default=100, help="Total number of time-points")
parser.add_argument('--max-t',  type=float, default=5., help="We subsample points in the interval [0, args.max_tp]")
parser.add_argument('--noise-weight', type=float, default=0.01, help="Noise amplitude for generated traejctories")




class Args:
    def __init__(self):
        self.n = 10000
        self.niters = 1000
        self.lr = 1e-2
        self.batch_size = 128
        self.viz = True
        self.save = 'experiments/'
        self.load = None
        self.random_seed = 1991
        self.dataset = 'physionet'
        self.sample_tp = 0.6
        self.cut_tp = None
        self.quantization = 5
        self.latent_ode = True
        self.z0_encoder = 'odernn'
        self.classic_rnn = False
        self.rnn_cell = "gru"
        self.input_decay = False
        self.ode_rnn = False
        self.rnn_vae = False
        self.latents = 20
        self.rec_dims = 10
        self.rec_layers = 5
        self.gen_layers = 5
        self.units = 50
        self.gru_units = 100
        self.poisson = False
        self.classif = False
        self.linear_classif = False
        self.extrap = False
        self.timepoints = 100
        self.max_t = 5.
        self.noise_weight = 0.01

# args 객체를 생성하고 필요한 설정을 할당합니다
args = Args()

# 몇 가지 설정을 수정합니다
args.batch_size = 64
args.classif = False
args.quantization = 5
args.niters = 30000
args.n = 1000
args.sample_tp = None
args.latents = 20
args.rec_dims = 10
args.rec_layers = 5
args.gen_layers = 5
args.latent_ode = True
args.viz = True
args.max_t= 3000


# 사용할 디바이스를 설정합니다
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 파일 이름을 설정합니다
file_name = 'taejun_sim_'

# 설정을 확인합니다
print("Batch size:", args.batch_size)
print("Number of iterations:", args.niters)
print("Learning rate:", args.lr)
print("Device:", device)



Batch size: 64
Number of iterations: 30000
Learning rate: 0.01
Device: cuda


In [16]:
args

In [17]:
import os
import torch
from tqdm import tqdm
from jdcal import jd2gcal
from datetime import datetime

class CustomClass(object):
    params = ['Magnitude']  # Uncertainty_of_Magnitude 제외

    params_dict = {k: i for i, k in enumerate(params)}

    def __init__(self, root, train=True, preprocess=False,
                 quantization=1, n_samples=None, device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")):
        self.root = root
        self.train = train
        self.quantization = quantization
        self.device = device
        self.reduce = "average"
        if preprocess:
            self.preprocess()

        self.data = torch.load(os.path.join(self.root, 'lc_' + str(self.quantization) + '.pt'))
        self.labels = torch.zeros(len(self.data))

        # if n_samples is not None:
        #     self.data = self.data[:n_samples]
        #     self.labels = self.labels[:n_samples]

    def preprocess(self):
        simulation_data_root = self.root
        data_list = [f for f in os.listdir(simulation_data_root) if f.endswith('.lc')]
        data_list.sort()
        
        # Process only up to 1000 files
        data_list = data_list[:1000]
        
        light_curves = []

        for name in tqdm(data_list):
            lc_name = name.partition('.')[0]
            file_path = os.path.join(simulation_data_root, name)
            
            # Step 1: Compute max and min values
            with open(file_path) as f:
                next(f)  # Skip header
                magnitudes = [float(line.rstrip().split(' ')[1]) for line in f]
                max_magnitude = max(magnitudes)
                min_magnitude = min(magnitudes)
            
            # Step 2: Process the file again to normalize and store data
            with open(file_path) as f:
                next(f)  # Skip header again
                lines = f.readlines()
                prev_time = 0
                tt = [0.]
                vals = [torch.zeros(len(self.params), device=self.device)]
                mask = [torch.zeros(len(self.params), device=self.device)]
                for line in lines:
                    time, magnitude = line.rstrip().split(' ')[:2]
                    time = float(time)
                    magnitude = float(magnitude)

                    # Normalize the magnitude
                    normalized_magnitude = (magnitude - min_magnitude) / (max_magnitude - min_magnitude)

                    if time != prev_time:
                        tt.append(time)
                        vals.append(torch.zeros(len(self.params), device=self.device))
                        mask.append(torch.zeros(len(self.params), device=self.device))
                        prev_time = time

                    vals[-1][0] = normalized_magnitude  # Use normalized value
                    mask[-1][0] = 1

            tt = torch.tensor(tt, device=self.device)[1:]
            vals = torch.stack(vals)[1:]
            mask = torch.stack(mask)[1:]
            labels = None
            light_curves.append((lc_name, tt, vals, mask, labels))

        torch.save(light_curves, os.path.join(self.root, 'lc_' + str(self.quantization) + '.pt'))
        print('Done!')


    # def preprocess(self):
    #     simulation_data_root = self.root 
    #     data_list = [f for f in os.listdir(simulation_data_root) if f.endswith('.txt')]
    #     data_list.sort()
    #     light_curves = []

    #     # 전체 데이터셋에 대한 최소값과 최대값을 초기화
    #     global_min = float('inf')
    #     global_max = float('-inf')

    #     # 최소값과 최대값 찾기
    #     for name in tqdm(data_list):
    #         with open(os.path.join(simulation_data_root, name)) as f:
    #             next(f)  # 헤더 건너뛰기
    #             for line in f:
    #                 _, magnitude = line.rstrip().split(' ')[:2]
    #                 magnitude = float(magnitude)
    #                 global_min = min(global_min, magnitude)
    #                 global_max = max(global_max, magnitude)

    #     # 데이터 스케일링 및 처리
    #     for name in tqdm(data_list):
    #         lc_name = name.partition('.')[0]
    #         with open(os.path.join(simulation_data_root, name)) as f:
    #             next(f)
    #             lines = f.readlines()
    #             prev_time = 0
    #             tt = [0.]
    #             vals = [torch.zeros(len(self.params), device=self.device)]
    #             mask = [torch.zeros(len(self.params), device=self.device)]
    #             for line in lines:
    #                 time, magnitude = line.rstrip().split(' ')[:2]
    #                 time = float(time)
    #                 magnitude = float(magnitude)

    #                 # 스케일링: (magnitude - global_min) / (global_max - global_min)
    #                 scaled_magnitude = (magnitude - global_min) / (global_max - global_min)

    #                 if time != prev_time:
    #                     tt.append(time)
    #                     vals.append(torch.zeros(len(self.params), device=self.device))
    #                     mask.append(torch.zeros(len(self.params), device=self.device))
    #                     prev_time = time

    #                 vals[-1][0] = scaled_magnitude  # 스케일된 값 사용
    #                 mask[-1][0] = 1

    #         tt = torch.tensor(tt, device=self.device)[1:]
    #         vals = torch.stack(vals)[1:]
    #         mask = torch.stack(mask)[1:]
    #         labels = None
    #         light_curves.append((lc_name, tt, vals, mask, labels))

    #     torch.save(light_curves, os.path.join(self.root, 'lc_' + str(self.quantization) + '.pt'))
    #     print('Done!')


    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        fmt_str = 'Dataset ' + self.__class__.__name__ + '\n'
        fmt_str += '    Number of datapoints: {}\n'.format(self.__len__())
        fmt_str += '    Split: {}\n'.format('train' if self.train else 'test')
        fmt_str += '    Root Location: {}\n'.format(self.root)
        fmt_str += '    Quantization: {}\n'.format(self.quantization)
        fmt_str += '    Reduce: {}\n'.format(self.reduce)
        return fmt_str

    def visualize(self, timesteps, data, mask, plot_name):
        width = 15
        height = 15
        timesteps.to
        non_zero_attributes = (torch.sum(mask,0) > 2).numpy()
        non_zero_idx = [i for i in range(len(non_zero_attributes)) if non_zero_attributes[i] == 1.]
        n_non_zero = sum(non_zero_attributes)

        mask = mask[:, non_zero_idx]
        data = data[:, non_zero_idx]

        params_non_zero = [self.params[i] for i in non_zero_idx]
        params_dict = {k: i for i, k in enumerate(params_non_zero)}

        n_col = 3
        n_row = n_non_zero // n_col + (n_non_zero % n_col > 0)
        fig, ax_list = plt.subplots(n_row, n_col, figsize=(width, height), facecolor='white')

        #for i in range(len(self.params)):
        for i in range(n_non_zero):
            param = params_non_zero[i]
            param_id = params_dict[param]

            tp_mask = mask[:,param_id].long()

            tp_cur_param = timesteps[tp_mask == 1.]
            data_cur_param = data[tp_mask == 1., param_id]

            ax_list[i // n_col, i % n_col].plot(tp_cur_param.numpy(), data_cur_param.numpy(),  marker='o') 
            ax_list[i // n_col, i % n_col].set_title(param)

        fig.tight_layout()
        fig.savefig(plot_name)
        plt.close(fig)



In [18]:
#코드에 구현된거 하지만 잘 안된다 수정해서 해보자
def variable_time_collate_fn(batch, args, device = device, data_type = "train", 
	data_min = None, data_max = None):
	D = batch[0][2].shape[1]
	combined_tt, inverse_indices = torch.unique(torch.cat([ex[1] for ex in batch]), sorted=True, return_inverse=True)
	combined_tt = combined_tt.to(device)

	offset = 0
	combined_vals = torch.zeros([len(batch), len(combined_tt), D]).to(device)
	combined_mask = torch.zeros([len(batch), len(combined_tt), D]).to(device)
	
	combined_labels = None
	N_labels = 1

	combined_labels = torch.zeros(len(batch), N_labels) + torch.tensor(float('nan'))
	combined_labels = combined_labels.to(device = device)
	
	for b, (record_id, tt, vals, mask, labels) in enumerate(batch):
		tt = tt.to(device)
		vals = vals.to(device)
		mask = mask.to(device)
		if labels is not None:
			labels = labels.to(device)

		indices = inverse_indices[offset:offset + len(tt)]
		offset += len(tt)

		combined_vals[b, indices] = vals
		combined_mask[b, indices] = mask

		if labels is not None:
			combined_labels[b] = labels

	combined_vals, _, _ = utils.normalize_masked_data(combined_vals, combined_mask, 
	 	att_min = data_min, att_max = data_max)

	if torch.max(combined_tt) != 0.:
		combined_tt = combined_tt / torch.max(combined_tt)
	
	data_dict = {
		"data": combined_vals, 
		"time_steps": combined_tt,
		"mask": combined_mask,
		"labels": combined_labels}
	
	data_dict = utils.split_and_subsample_batch(data_dict, args, data_type = data_type)
	return data_dict

In [19]:
train_dataset_obj = CustomClass(root='/home/intern/SSD/intern/taejun/r_light_curves', train=True, preprocess=True,quantization=5,n_samples =  args.n, device=device)
		# Use custom collate_fn to combine samples with arbitrary time observations.
		# Returns the dataset along with mask and time steps


# Combine and shuffle samples from physionet Train and physionet Test
total_dataset = train_dataset_obj[:len(train_dataset_obj)]



# Shuffle and split
train_data, test_data = model_selection.train_test_split(total_dataset, train_size= 0.8, 
    random_state = 42, shuffle = True)

record_id, tt, vals, mask, labels = train_data[0]

n_samples = len(total_dataset)
input_dim = vals.size(-1)

batch_size = min(min(len(train_dataset_obj), args.batch_size), args.n)
data_min, data_max = get_data_min_max(total_dataset)

train_dataloader = DataLoader(train_data, batch_size= batch_size, shuffle=False, 
    collate_fn= lambda batch: variable_time_collate_fn(batch, args, device, data_type = "train",
        data_min = data_min, data_max = data_max))
test_dataloader = DataLoader(test_data, batch_size = batch_size, shuffle=False, 
    collate_fn= lambda batch: variable_time_collate_fn(batch, args, device, data_type = "train",
        data_min = data_min, data_max = data_max))

attr_names = train_dataset_obj.params
data_objects = {"dataset_obj": train_dataset_obj, 
            "train_dataloader": utils.inf_generator(train_dataloader), 
            "test_dataloader": utils.inf_generator(test_dataloader),
            "input_dim": input_dim,
            "n_train_batches": len(train_dataloader),
            "n_test_batches": len(test_dataloader),
            "attr": attr_names, #optional
            }


100%|██████████| 1000/1000 [00:56<00:00, 17.84it/s]


Done!


In [20]:
args.sample_tp

In [21]:
# pt_file_path = '/home/intern/SSD/intern/taejun/data_normal/lc_5.pt'

# # .pt 파일 로드
# light_curves_data = torch.load(pt_file_path)

# # 로드된 데이터의 타입과 크기 확인
# print(f"Loaded data type: {type(light_curves_data)}")
# print(f"Number of light curves in the dataset: {len(light_curves_data)}")

# # 첫 번째 광도곡선 데이터의 구조 확인
# first_light_curve = light_curves_data[0]
# print(f"Structure of a single light curve data: {type(first_light_curve)}")
# print(f"Record ID: {first_light_curve[0]}")
# print(f"Time stamps tensor shape: {first_light_curve[1].shape}")
# print(f"Magnitude tensor shape: {first_light_curve[2].shape}")
# print(f"Mask tensor shape: {first_light_curve[3].shape}")
# #print(f"Labels tensor shape: {first_light_curve[4].shape}")

# # 첫 번째 광도곡선 데이터의 일부 내용 출력
# print("\nSample data from the first light curve:")
# print(f"Time stamps: {first_light_curve[1]}")  # 처음 5개의 시간 스탬프
# print(f"Magnitude: {first_light_curve[2]}")  # 처음 5개의 광도 값


In [22]:
# train_data

In [23]:
# test_data

In [24]:
# `train_dataloader`에서 첫 번째 배치를 가져와서 형태와 내용을 확인합니다.
print("Train DataLoader:")
for i, batch in enumerate(train_dataloader):
    print(f"Batch {i+1}:")
    print("observed_data.shape:", batch['observed_data'].shape)
    #print("observed_data:", batch['observed_data'])
    print("data_to_predict:", batch['data_to_predict'].shape)
    #여기에서 필요한 다른 키들도 확인할 수 있습니다.
    # 예를


Train DataLoader:
Batch 1:
observed_data.shape: torch.Size([64, 199, 1])
data_to_predict: torch.Size([64, 199, 1])
Batch 2:
observed_data.shape: torch.Size([64, 200, 1])
data_to_predict: torch.Size([64, 200, 1])
Batch 3:
observed_data.shape: torch.Size([64, 200, 1])
data_to_predict: torch.Size([64, 200, 1])
Batch 4:
observed_data.shape: torch.Size([64, 193, 1])
data_to_predict: torch.Size([64, 193, 1])
Batch 5:
observed_data.shape: torch.Size([64, 199, 1])
data_to_predict: torch.Size([64, 199, 1])
Batch 6:
observed_data.shape: torch.Size([64, 201, 1])
data_to_predict: torch.Size([64, 201, 1])
Batch 7:
observed_data.shape: torch.Size([64, 205, 1])
data_to_predict: torch.Size([64, 205, 1])
Batch 8:
observed_data.shape: torch.Size([64, 191, 1])
data_to_predict: torch.Size([64, 191, 1])
Batch 9:
observed_data.shape: torch.Size([64, 195, 1])
data_to_predict: torch.Size([64, 195, 1])
Batch 10:
observed_data.shape: torch.Size([64, 204, 1])
data_to_predict: torch.Size([64, 204, 1])
Batch 11:
o

In [25]:
import os

# Path to the directory where you want to save the file
directory = "experiments"

# Check if the directory exists
if not os.path.exists(directory):
    # If the directory does not exist, create it
    os.makedirs(directory)

In [26]:


experimentID = args.load if args.load is not None else int(SystemRandom().random() * 100000)
ckpt_path = os.path.join(args.save, f"experiment_{experimentID}.ckpt")

# 데이터 로딩 및 처리
data_obj = data_objects # 'data_objects'는 데이터 로딩 및 전처리를 담당하는 코드 부분에서 정의됨
input_dim = data_obj["input_dim"]

# 모델 생성
obsrv_std = torch.Tensor([0.01]).to(device)
z0_prior = Normal(torch.Tensor([0.0]).to(device), torch.Tensor([1.]).to(device))

if args.latent_ode:
    model = create_LatentODE_model(args, input_dim, z0_prior, obsrv_std, device,
                                   classif_per_tp=False, # 이 예제에서는 간소화를 위해 False로 설정
                                   n_labels=1) # 이 예제에서는 단순화를 위해 1로 설정
else:
    raise Exception("Model not specified")

# # 로그 설정
# log_path = f"logs/{file_name}_{experimentID}.log"
# if not os.path.exists("logs/"):
#     os.makedirs("logs/")
# logger = utils.get_logger(logpath=log_path, filepath=os.path.abspath(__file__))
# logger.info(" ".join(sys.argv))

# 최적화기 설정
optimizer = optim.Adam(model.parameters(), lr=args.lr)

# 훈련 과정
num_batches = data_obj["n_train_batches"]
for itr in range(1, num_batches * (args.niters + 1)):
    optimizer.zero_grad()
    utils.update_learning_rate(optimizer, decay_rate=0.999, lowest=args.lr / 10)

    kl_coef = 0. if itr // num_batches < 10 else (1-0.99 ** (itr // num_batches - 10))

    batch_dict = utils.get_next_batch(data_obj["train_dataloader"])
    train_res = model.compute_all_losses(batch_dict, n_traj_samples=10, kl_coef=kl_coef)
    train_res["loss"].backward()
    optimizer.step()

    # 주기적 로깅
    if itr % (10 * num_batches) == 0:
        print(f"Epoch {itr // num_batches} | Loss {train_res['loss'].item()} | Likelihood {train_res['likelihood'].item()} | KL Coef {kl_coef}")

        # 훈련 데이터 시각화 (선택적)
        if args.viz:  # 시각화 옵션이 활성화된 경우
            plot_name_train = f"final_real_lab_data/train_finalresult_20_{itr // num_batches:04d}"
            with torch.no_grad():
                Visualizations(device).draw_all_plots_one_dim(batch_dict, model.to(device), plot_name=plot_name_train, save=True)
            with torch.no_grad():
                test_res = compute_loss_all_batches(model, 
					data_obj["test_dataloader"], args,
					n_batches = data_obj["n_test_batches"],
					experimentID = experimentID,
					device = device,
					n_traj_samples = 10, kl_coef = kl_coef)
                sample_test = utils.get_next_batch(data_obj["test_dataloader"])
                plot_name_test = f"final_real_lab_data/test_finalresult_20_{itr // num_batches:04d}"
                Visualizations(device).draw_all_plots_one_dim(sample_test ,model.to(device), plot_name=plot_name_test, save = True)
				

# 체크포인트 저장 (선택적)
# torch.save(model.state_dict(), ckpt_path)


Epoch 10 | Loss 211.82325744628906 | Likelihood -215.18516540527344 | KL Coef 0.0


/media/usr/SSD/intern/taejun/latent_ode/lib/plotting.py:164: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  self.fig = plt.figure(figsize=(20, 15), facecolor='white',constrained_layout=True)


Computing loss... 0
Computing loss... 1
Computing loss... 2
Computing loss... 3
Epoch 20 | Loss 123.16305541992188 | Likelihood -125.56639862060547 | KL Coef 0.09561792499119559
Computing loss... 0
Computing loss... 1
Computing loss... 2
Computing loss... 3
Epoch 30 | Loss 112.76026153564453 | Likelihood -115.2221908569336 | KL Coef 0.18209306240276923
Computing loss... 0
Computing loss... 1
Computing loss... 2
Computing loss... 3
Epoch 40 | Loss 121.59784698486328 | Likelihood -123.54615783691406 | KL Coef 0.2602996266117198
Computing loss... 0
Computing loss... 1
Computing loss... 2
Computing loss... 3
Epoch 50 | Loss 106.22374725341797 | Likelihood -108.4842758178711 | KL Coef 0.3310282414303197
Computing loss... 0
Computing loss... 1
Computing loss... 2
Computing loss... 3
Epoch 60 | Loss 104.63167572021484 | Likelihood -107.0267562866211 | KL Coef 0.39499393286246365
Computing loss... 0
Computing loss... 1
Computing loss... 2
Computing loss... 3
Epoch 70 | Loss 91.68841552734375 |

ValueError: Expected parameter scale (Tensor of shape (1, 64, 20)) of distribution Normal(loc: torch.Size([1, 64, 20]), scale: torch.Size([1, 64, 20])) to satisfy the constraint GreaterThan(lower_bound=0.0), but found invalid values:
tensor([[[0.1346, 0.0862, 0.0352,  ..., 0.2907, 0.5341, 0.0526],
         [0.2618, 0.0149, 0.2634,  ..., 0.4181, 0.9398, 0.0369],
         [0.0555, 0.0463, 0.0560,  ..., 0.5011, 0.7055, 0.0220],
         ...,
         [0.2551, 0.0506, 0.5747,  ..., 0.1992, 0.8548, 0.0110],
         [0.1518, 0.0059, 0.1649,  ..., 0.4848, 0.7787, 0.0366],
         [0.0592, 0.0796, 0.2870,  ..., 0.5021, 0.6303, 0.0081]]],
       device='cuda:0', grad_fn=<AbsBackward0>)

In [27]:
with torch.no_grad():
    final_test_res = compute_loss_all_batches(model, 
                    data_obj["test_dataloader"], args,
                    n_batches=data_obj["n_test_batches"],
                    experimentID=experimentID,
                    device=device,
                    n_traj_samples=10, kl_coef=kl_coef)

    print("Final Test Loss Summary:")
    for key, value in final_test_res.items():
        print(f"{key}: {value}")

Computing loss... 0
Computing loss... 1
Computing loss... 2
Computing loss... 3
Final Test Loss Summary:
loss: 81.40855407714844
likelihood: -81.25747680664062
mse: 0.016988741233944893
kl_first_p: 2.939375400543213
std_first_p: 0.22200098633766174
pois_likelihood: 0.0
ce_loss: 0.0


In [ ]:
# if __name__ == '__main__':
# 	torch.manual_seed(args.random_seed)
# 	np.random.seed(args.random_seed)

# 	experimentID = args.load
# 	if experimentID is None:
# 		# Make a new experiment ID
# 		experimentID = int(SystemRandom().random()*100000)
# 	ckpt_path = os.path.join(args.save, "experiment_" + str(experimentID) + '.ckpt')

# 	start = time.time()
# 	print("Sampling dataset of {} training examples".format(args.n))
	
# 	input_command = sys.argv
# 	ind = [i for i in range(len(input_command)) if input_command[i] == "--load"]
# 	if len(ind) == 1:
# 		ind = ind[0]
# 		input_command = input_command[:ind] + input_command[(ind+2):]
# 	input_command = " ".join(input_command)

# 	utils.makedirs("results/")

# 	##################################################################
# 	data_obj = data_objects
# 	input_dim = data_obj["input_dim"]

# 	classif_per_tp = False
# 	if ("classif_per_tp" in data_obj):
# 		# do classification per time point rather than on a time series as a whole
# 		classif_per_tp = data_obj["classif_per_tp"]

# 	if args.classif and (args.dataset == "hopper" or args.dataset == "periodic"):
# 		raise Exception("Classification task is not available for MuJoCo and 1d datasets")

# 	n_labels = 1
# 	if args.classif:
# 		if ("n_labels" in data_obj):
# 			n_labels = data_obj["n_labels"]
# 		else:
# 			raise Exception("Please provide number of labels for classification task")

# 	##################################################################
# 	# Create the model
# 	obsrv_std = 0.01
# 	obsrv_std = torch.Tensor([obsrv_std]).to(device)
# 	z0_prior = Normal(torch.Tensor([0.0]).to(device), torch.Tensor([1.]).to(device))

# 	if args.rnn_vae:
# 		if args.poisson:
# 			print("Poisson process likelihood not implemented for RNN-VAE: ignoring --poisson")

# 		# Create RNN-VAE model
# 		model = RNN_VAE(input_dim, args.latents, 
# 			device = device, 
# 			rec_dims = args.rec_dims, 
# 			concat_mask = True, 
# 			obsrv_std = obsrv_std,
# 			z0_prior = z0_prior,
# 			use_binary_classif = args.classif,
# 			classif_per_tp = classif_per_tp,
# 			linear_classifier = args.linear_classif,
# 			n_units = args.units,
# 			input_space_decay = args.input_decay,
# 			cell = args.rnn_cell,
# 			n_labels = n_labels,
# 			train_classif_w_reconstr = (args.dataset == "physionet")
# 			).to(device)
		
# 	elif args.classic_rnn:
# 		if args.poisson:
# 			print("Poisson process likelihood not implemented for RNN: ignoring --poisson")

# 		if args.extrap:
# 			raise Exception("Extrapolation for standard RNN not implemented")
# 		# Create RNN model
# 		model = Classic_RNN(input_dim, args.latents, device, 
# 			concat_mask = True, obsrv_std = obsrv_std,
# 			n_units = args.units,
# 			use_binary_classif = args.classif,
# 			classif_per_tp = classif_per_tp,
# 			linear_classifier = args.linear_classif,
# 			input_space_decay = args.input_decay,
# 			cell = args.rnn_cell,
# 			n_labels = n_labels,
# 			train_classif_w_reconstr = (args.dataset == "physionet")
# 			).to(device)
		
# 	elif args.ode_rnn:
# 		# Create ODE-GRU model
# 		n_ode_gru_dims = args.latents
				
# 		if args.poisson:
# 			print("Poisson process likelihood not implemented for ODE-RNN: ignoring --poisson")

# 		if args.extrap:
# 			raise Exception("Extrapolation for ODE-RNN not implemented")

# 		ode_func_net = utils.create_net(n_ode_gru_dims, n_ode_gru_dims, 
# 			n_layers = args.rec_layers, n_units = args.units, nonlinear = nn.Tanh)

# 		rec_ode_func = ODEFunc(
# 			input_dim = input_dim, 
# 			latent_dim = n_ode_gru_dims,
# 			ode_func_net = ode_func_net,
# 			device = device).to(device)

# 		z0_diffeq_solver = DiffeqSolver(input_dim, rec_ode_func, "euler", args.latents, 
# 			odeint_rtol = 1e-3, odeint_atol = 1e-4, device = device)
	
# 		model = ODE_RNN(input_dim, n_ode_gru_dims, device = device, 
# 			z0_diffeq_solver = z0_diffeq_solver, n_gru_units = args.gru_units,
# 			concat_mask = True, obsrv_std = obsrv_std,
# 			use_binary_classif = args.classif,
# 			classif_per_tp = classif_per_tp,
# 			n_labels = n_labels,
# 			train_classif_w_reconstr = (args.dataset == "physionet")
# 			).to(device)
# 	elif args.latent_ode:
# 		model = create_LatentODE_model(args, input_dim, z0_prior, obsrv_std, device, 
# 			classif_per_tp = classif_per_tp,
# 			n_labels = n_labels)
# 	else:
# 		raise Exception("Model not specified")

# 	##################################################################

# 	if args.viz:
# 		viz = Visualizations(device)

# 	##################################################################
	
# 	#Load checkpoint and evaluate the model
# 	if args.load is not None:
# 		utils.get_ckpt_model(ckpt_path, model, device)
# 		exit()

# 	##################################################################
# 	# Training

# 	log_path = "logs/" + file_name + "_" + str(experimentID) + ".log"
# 	if not os.path.exists("logs/"):
# 		utils.makedirs("logs/")
# 	logger = utils.get_logger(logpath=log_path, filepath=os.path.abspath(file_name))
# 	logger.info(input_command)

# 	optimizer = optim.Adamax(model.parameters(), lr=args.lr)

# 	num_batches = data_obj["n_train_batches"]

# 	for itr in range(1, num_batches * (args.niters + 1)):
# 		optimizer.zero_grad()
# 		utils.update_learning_rate(optimizer, decay_rate=0.999, lowest=args.lr / 10)

# 		wait_until_kl_inc = 10
# 		if itr // num_batches < wait_until_kl_inc:
# 			kl_coef = 0.
# 		else:
# 			kl_coef = (1-0.99**(itr // num_batches - wait_until_kl_inc))

# 		batch_dict = utils.get_next_batch(data_obj["train_dataloader"])
# 		print(batch_dict['observed_data'])
# 		train_res = model.compute_all_losses(batch_dict, n_traj_samples=10, kl_coef=kl_coef)
# 		train_res["loss"].backward()
# 		optimizer.step()

# 		# 주기적으로 로깅 및 시각화를 위한 조건문
# 		if itr % (10 * num_batches) == 0:  # 예를 들어, 10 epoch 마다 로그와 시각화 수행
# 			logger.info(f"Epoch {itr // num_batches} | Loss {train_res['loss'].item()} | Likelihood {train_res['likelihood'].item()} | KL Coef {kl_coef}")

# 			# 훈련 데이터 시각화
# 			plot_name_train = f"train_result_{itr // num_batches:04d}"
# 			with torch.no_grad():  # 시각화 동안에는 기울기 계산을 하지 않음
# 				Visualizations(device).draw_all_plots_one_dim(batch_dict, model.to(device), plot_name=plot_name_train, save=True)

# 			# 추가 정보 로깅
# 			logger.info(f"Train loss (one batch): {train_res['loss'].item()}")
# 			if 'ce_loss' in train_res:
# 				logger.info(f"Train CE loss (one batch): {train_res['ce_loss'].item()}")


				

			


			



In [ ]:
batch_dict['observed_data'].shape

torch.Size([4, 523, 1])

In [ ]:
data_obj["test_dataloader"]

<generator object inf_generator at 0x7fcfd36ec190>